In [18]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run.split('_')[0]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/join_ssd'
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'iostat.bytes_read', 'iostat.bytes_wrtn',
       'spec.index.epsilon', 'spec.index.leaf_size_in_pages',
       'spec.index.search', 'spec.index.type'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk,iostat.bytes_read,iostat.bytes_wrtn,spec.index.epsilon,spec.index.leaf_size_in_pages,spec.index.search,spec.index.type
0,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,45841991062,45.841991,781250,4096,3199999984,1989865,0,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/hash_join...,8,True,3162272000,28040000,NaN,NaN,NaN,NaN
1,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,6972632156,6.972632,781250,4096,3199999984,1989865,7773,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/sort_join...,8,True,3161960000,64000,NaN,NaN,NaN,NaN
2,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,30721282928,30.721283,781250,4096,3199999984,199999999,781250,...,sponge/join_ssd/fb_threads=1/inputs/input1,sponge/join_ssd/fb_threads=1/outputs/sort_join...,8,True,6255952000,2565744000,NaN,NaN,NaN,NaN
3,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,82335846276,82.335846,781250,4096,3199999984,199999999,0,...,sponge/join_ssd/fb_threads=1/inputs/input1,sponge/join_ssd/fb_threads=1/outputs/hash_join...,8,True,6256952000,3094308000,NaN,NaN,NaN,NaN
4,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,46073187735,46.073188,781250,4096,3199999984,1989865,0,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/hash_join...,8,True,3163004000,29896000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,51117079600,51.117080,781250,4096,3199999984,199999999,781250,...,sponge/join_ssd/fb_threads=1/inputs/input1,sponge/join_ssd/fb_threads=1/outputs/inlj_btre...,8,True,6264516000,2479148000,256.0,1.0,binary,btree
68,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,6865628788,6.865629,720890,4096,2952765424,1989865,7773,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/lsj_btree...,8,True,3169920000,11284000,256.0,1.0,binary,btree
69,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,6726388258,6.726388,720890,4096,2952765424,1989865,7773,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/inlj_btre...,8,True,3169980000,68000,256.0,1.0,binary,btree
70,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,6632519850,6.632520,386319,8192,3164721136,1989865,7773,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/lsj_sampl...,8,True,3161140000,64000,256.0,NaN,binary,sampledflatpgm256


In [19]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['disk_read_gb'] = df['iostat.bytes_read']/1e9
df['disk_wrtn_gb'] = df['iostat.bytes_wrtn']/1e9
#df['cdisk_read_gb'] = df['cgroup_iostat.bytes_read']/1e9
#df['cdisk_wrtn_gb'] = df['cgroup_iostat.bytes_wrtn']/1e9
df['checksum'] = df['result.checksum']
df['threads'] = df['spec.num_threads']

df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")


┌──────────────────────┬─────────┬─────────────────┬────────────────────┬───┬──────────────┬──────────────┬──────────┐
│       command        │ dataset │ result.checksum │ result.duration_ns │ … │ disk_read_gb │ disk_wrtn_gb │ checksum │
│       varchar        │ varchar │      int64      │       int64        │   │    double    │    double    │  int64   │
├──────────────────────┼─────────┼─────────────────┼────────────────────┼───┼──────────────┼──────────────┼──────────┤
│ sponge/join_ssd/bu…  │ fb      │               0 │        45841991062 │ … │     3.162272 │      0.02804 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │         6972632156 │ … │      3.16196 │      6.4e-05 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │        30721282928 │ … │     6.255952 │     2.565744 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │        82335846276 │ … │     6.256952 │     3.094308 │        0 │
│ sponge/join_ssd/bu…  │ fb      │              

In [26]:
def print_for_dataset(dataset, algos):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    rows[algos].to_csv('/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/join_ssd/'+dataset+'.txt')



algos = ['lsj_sampledflatpgm256', 'inlj_btree256',  'sort_join', 'hash_join']
print_for_dataset("fb", algos)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)



## fb
|   ratio |   lsj_sampledflatpgm256 |   inlj_btree256 |   sort_join |   hash_join |
|--------:|------------------------:|----------------:|------------:|------------:|
|       1 |                32.6124  |        49.3887  |    31.2993  |     80.9313 |
|      10 |                 7.77427 |         8.00319 |     8.7249  |     55.2928 |
|     100 |                 6.63252 |         6.72639 |     7.17717 |     45.842  |
|    1000 |                18.0357  |        22.4017  |     7.08269 |     21.0575 |


In [20]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='books'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='osm'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))



algo,hash_join,inlj_btree256,inlj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm256,sort_join
ratio,,,,,,
1,80.931304,49.388665,44.882598,55.784585,32.612449,31.299311
10,55.292828,8.003186,8.061300,8.363397,7.774270,8.724903
100,45.841991,6.726388,6.664628,6.787620,6.632520,7.177175
1000,21.057477,22.401654,18.125491,22.804902,18.035658,7.082692


algo,lsj_sampledflatpgm256
ratio,
1,32.612449
10,7.774270
100,6.632520
1000,18.035658


algo
ratio


algo
ratio


algo
ratio


algo
ratio


In [21]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))


algo
ratio


algo
ratio


algo
ratio


In [8]:
def print_for_dataset(dataset, algos):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    rows[algos].to_csv('all_indexes/'+dataset+'.csv')



algos = ['lsj_sampledflatpgm256', 'lsj_btree256',  'sort_join', 'hash_join']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)



## fb


BinderException: Binder Error: Referenced column "threads" not found in FROM clause!
Candidate bindings: "df.dataset"

In [34]:
algos = ['lsj_sampledflatpgm256', 'lsj_flatpgm256', 'lsj_radixspline256', 'lsj_rmi' , 'lsj_btree256', 'sort_join']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)


## fb
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_rmi |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|----------:|---------------:|------------:|
|       1 |               23.2294   |        22.4319   |            25.6702   | 20.3287   |       42.3009  |    11.7042  |
|      10 |                3.68178  |         3.45073  |             4.30738  |  3.76312  |        5.83421 |     4.45029 |
|     100 |                0.98362  |         1.01192  |             1.46279  |  1.25237  |        1.57196 |     3.12833 |
|    1000 |                0.385151 |         0.353513 |             0.374326 |  0.275802 |        0.31908 |     2.86744 |
## wiki
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|---------------:|------------:|
|       1 |               

In [29]:
algos = ['lsj_radixspline256', 'lsj_radixspline1024', 'lsj_radixspline4096']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)

## fb


BinderException: Binder Error: Referenced column "threads" not found in FROM clause!
Candidate bindings: "df.dataset"